# [Day-02](https://adventofcode.com/2019/day/2), [Day-05](https://adventofcode.com/2019/day/5), [Day-07](https://adventofcode.com/2019/day/7), [Day-09](https://adventofcode.com/2019/day/9): Intcode Program Interpreter

In [111]:
from operator import add, mul, lt, eq
from copy import copy

def exec_intcode_program(program, op_pointer=0, inputs=None):
    """ Executes the intcode program, used in days 2, 5, 7 & 9 of Advent of Code 2019. 
        - program: The program to execute.
        - op_pointer: The pointer to the op to start the program at.
        - inputs: The set of inputs for the program when it is requested. 
        The program will be adjusted, and will stop when requested to (op=99),
        or when requested for input (op=3) without available input.
        """
    # Define the relative base of the operations.
    relative_base = 0
    def change_relative_base(change): 
        nonlocal relative_base 
        relative_base += change
    # Define the program operations.
    opcode_details = {
        1: {"in_params_count": 2, "store_output": 1, "op": add}, 
        2: {"in_params_count": 2, "store_output": 1, "op": mul}, 
        3: {"in_params_count": 0, "store_output": 1, "op": lambda: inputs.pop(0)}, 
        4: {"in_params_count": 1, "store_output": 0, "op": lambda x: outputs.append(x)},
        5: {"in_params_count": 2, "store_output": 0, "op": lambda cond, addr: addr if cond else None},
        6: {"in_params_count": 2, "store_output": 0, "op": lambda cond, addr: None if cond else addr},
        7: {"in_params_count": 2, "store_output": 1, "op": lt},
        8: {"in_params_count": 2, "store_output": 1, "op": eq},
        9: {"in_params_count": 1, "store_output": 0, "op": change_relative_base}
    }
    # Define the logic to converting parameters to their values' addresses from the param mode.
    def get_param_value_address(param_address, param_mode):
        if   param_mode == 0: return program[param_address]
        elif param_mode == 1: return param_address
        elif param_mode == 2: return program[param_address] + relative_base
    # Start the program.
    outputs = []
    while True:
        # Break down op pointer to op instruction parts.
        op_instruction = str(program[op_pointer]).zfill(5)
        opcode, param_modes = int(op_instruction[-2:]), [int(digit) for digit in op_instruction[-3::-1]]
        # Make sure we don't need to stop the program.
        if opcode == 99 or (opcode == 3 and not inputs):
            return opcode, op_pointer, outputs 
        # Retrieve opcode details from opcode.
        in_params_count = opcode_details[opcode]["in_params_count"]
        store_output    = opcode_details[opcode]["store_output"]
        op              = opcode_details[opcode]["op"]
        # Retrieve parameter value address from the param mode.
        in_params_values = [program[get_param_value_address(param_address, param_mode)] for param_address, param_mode
                            in zip(range(op_pointer+1, op_pointer+1+in_params_count), param_modes[:in_params_count])]
        # Execute the operation.
        out_val = op(*in_params_values)
        # Store the output in the out_param specified address if required.
        if store_output:
            out_address = get_param_value_address(param_address=op_pointer + 1 + in_params_count,
                                                  param_mode=param_modes[in_params_count])
            program[out_address] = out_val
        # Advance the op pointer (jump if successful jump command, advance otherwise)
        op_pointer = (out_val if (opcode in [5,6] and out_val is not None) 
                      else op_pointer + 1 + in_params_count + store_output)

## Day-02 - find program[0] value for different noun & verb

In [112]:
from itertools import product

with open("./input/Day-02.txt", "r") as program_file:
    program_02 = [int(x) for x in program_file.readline().strip().split(',')]

def set_noun_verb_and_exec_intcode_program(program:list, noun:int, verb:int):
    program = copy(program)
    program[1] = noun
    program[2] = verb
    exec_intcode_program(program)
    return program[0]

def search_noun_verb_from_result(program, result):
    for noun, verb in product(range(100), range(100)):
        if set_noun_verb_and_exec_intcode_program(program, noun, verb) == result:
            return noun * 100 + verb

print(f"2a. program[0] for [1202] program is [{set_noun_verb_and_exec_intcode_program(program_02, noun=12, verb=2)}].")
print(f"2b. program[0] for [{search_noun_verb_from_result(program_02, 19690720)}] program is [19690720].")

2a. program[0] for [1202] program is [3790689].
2b. program[0] for [6533] program is [19690720].


# Day-05 - operators 3-8 + parameters modes

In [113]:
with open("./input/Day-05.txt", "r") as program_file:
    program_05 = [int(x) for x in program_file.readline().strip().split(',')]

print("5a. Intcode program outputs (input=1):", exec_intcode_program(copy(program_05), inputs=[1])[2])
print("5b. Intcode program outputs (input=5):", exec_intcode_program(copy(program_05), inputs=[5])[2])

5a. Intcode program outputs (input=1): [0, 0, 0, 0, 0, 0, 0, 0, 0, 10987514]
5b. Intcode program outputs (input=5): [14195011]


## Day-07 - amplifying the signal

In [114]:
from itertools import permutations

with open("./input/Day-07.txt", "r") as program_file:
    program_07 = [int(x) for x in program_file.readline().strip().split(',')]

def find_highest_signal(program, possible_settings):
    highest_signal, best_setting = 0, None
    # Try all permutations of the possible settings.
    for permutation in permutations(possible_settings):
        # Keep separate copies of the programs for each amplifier.
        programs = dict((setup, copy(program)) for setup in permutation)
        op_pointers = dict((setup, 0) for setup in permutation)
        # Setup the initial input signal for each amplifier.
        inputs = dict((setup, [setup]) for setup in permutation)
        inputs[permutation[0]].append(0)
        # Execute all amplifiers, feeding one amplifier's output to the next's input. 
        i = 0
        while True:
            # Execute the current setup's amplifier.
            setup = permutation[i % len(permutation)]
            opcode, op_pointers[setup], output = exec_intcode_program(
                programs[setup], op_pointer=op_pointers[setup], inputs=inputs[setup])               
            # Add the amplifier's output to the next amplifier's input.
            next_setup = permutation[(i+1) % len(permutation)]
            inputs[next_setup] += output
            # Stop if we've finished executing the last amplifier. 
            if opcode == 99 and not (i + 1) % len(permutation):
                output = output.pop()
                break
            # Otherwise, advance to the next amplifier.
            i += 1
        # Keep track of the best settings.        
        if output > highest_signal:
            highest_signal = output
            best_setting = permutation
    return highest_signal, best_setting

print("7a. Highest signal is [{}] with the setup {}.".format(*find_highest_signal(program_07, range(5))))
print("7b. Highest signal is [{}] with the setup {}.".format(*find_highest_signal(program_07, range(5,10))))

7a. Highest signal is [14902] with the setup (3, 1, 2, 0, 4).
7b. Highest signal is [6489132] with the setup (9, 6, 7, 5, 8).


## Day-09 - relative base parameter mode.

In [115]:
from collections import defaultdict

with open("./input/Day-09.txt", "r") as program_file:
    program_09 = [int(x) for x in program_file.readline().strip().split(',')]
    program_09 = defaultdict(lambda: 0, zip(range(len(program_09)), program_09))  # For unlimited memory address.

print("9a. Intcode program outputs (input=1):", exec_intcode_program(copy(program_09), inputs=[1])[2])
print("9b. Intcode program outputs (input=2):", exec_intcode_program(copy(program_09), inputs=[2])[2])

9a. Intcode program outputs (input=1): [2932210790]
9b. Intcode program outputs (input=2): [73144]
